Rappel du format des données : <br>
mains_3J = [preflop, flop, turn, river, proba, button, small, bigblind,BB, my_stack, stack_J2, stack_J3, my_bet, bet_J2, bet_J3, pot, decision, decision_value]<br>

## Récupération des mains à partir des fichiers

In [1]:
import numpy as np
import torch

import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


BATCH_SIZE = 1024

In [2]:
mains_3J = np.load('mains3J.npy')

In [3]:
#creation d'un mask pour detecter les all in ( > 4)
list_allin_3J = np.where(mains_3J[:,17] > 4)[0]

# On supprimer les all-ins : ils seront gérés par un tableau de psuh or fold
mains_3J = np.delete(mains_3J, list_allin_3J, 0)

## Séparation des données

In [4]:
nb_features = mains_3J.shape[1]
mains_3J_X = mains_3J[:,:(nb_features-2)]
mains_3J_Y = mains_3J[:, (nb_features-2):(nb_features-1)].squeeze()

scaler = StandardScaler()
scaler.fit(mains_3J_X[:,[4,8,9,10,11,12,13,14,15]])

mains_3J_X[:,[4,8,9,10,11,12,13,14,15]] = scaler.transform(mains_3J_X[:,[4,8,9,10,11,12,13,14,15]])

mains_3J_X_train, mains_3J_X_val, mains_3J_Y_train, mains_3J_Y_val = train_test_split(mains_3J_X, mains_3J_Y,
                                                                     test_size=0.01, stratify=mains_3J_Y)


In [5]:
np.savez('mean_var_3J.npz', mean=scaler.mean_, var=scaler.var_)

## Datasets et dataloaders

In [40]:
class MainsDataset():
    def __init__(self, X, Y):
        self.hands= torch.from_numpy(X).type(torch.cuda.DoubleTensor)
        self.labels= torch.from_numpy(Y).type(torch.cuda.LongTensor)
        
    def __getitem__(self, index):
        hand =  self.hands[index]
        label =  self.labels[index]
        return (hand, label)

    def __len__(self):
        count = len(self.hands)
        return count 

In [41]:
train_mains_3J = MainsDataset(mains_3J_X_train, mains_3J_Y_train)
train_mains_3J_loader = torch.utils.data.DataLoader(train_mains_3J, batch_size=BATCH_SIZE, shuffle=True)

val_mains_3J = MainsDataset(mains_3J_X_val, mains_3J_Y_val)
val_mains_3J_loader = torch.utils.data.DataLoader(val_mains_3J, batch_size=BATCH_SIZE, shuffle=True)


## Construction du réseau de neurones

In [42]:
class NN_3J(nn.Module):

    def __init__(self):
        super(NN_3J, self).__init__()
        self.largeur = 500
                
        self.input = nn.Linear(16, self.largeur)
        self.hidden = nn.Linear(self.largeur, self.largeur)
        self.final = nn.Linear(self.largeur, 3)


    def forward(self, x):

        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden(x))
        x = self.final(x)
        return x

NN_3J = NN_3J()
print(NN_3J)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NN_3J.to(device).type(torch.cuda.DoubleTensor)

NN_3J(
  (input): Linear(in_features=16, out_features=500, bias=True)
  (hidden): Linear(in_features=500, out_features=500, bias=True)
  (final): Linear(in_features=500, out_features=3, bias=True)
)


NN_3J(
  (input): Linear(in_features=16, out_features=500, bias=True)
  (hidden): Linear(in_features=500, out_features=500, bias=True)
  (final): Linear(in_features=500, out_features=3, bias=True)
)

## Entrainement du réseau 

In [43]:

criterion = nn.CrossEntropyLoss()

lr_tab = [0.1, 0.08, 0.06, 0.04, 0.02, 0.01]


for lr_ in lr_tab:
    
    optimizer = optim.SGD(NN_3J.parameters(), lr=lr_)
    for epoch in range(25):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_mains_3J_loader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = NN_3J(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('lr = %.3f  |  %d loss: %.3f' %(lr_ ,epoch + 1,running_loss*BATCH_SIZE))

print('Finished Training')

lr = 0.100  |  1 loss: 53907.515
lr = 0.100  |  2 loss: 43802.865
lr = 0.100  |  3 loss: 37196.978
lr = 0.100  |  4 loss: 33695.614
lr = 0.100  |  5 loss: 31862.292
lr = 0.100  |  6 loss: 30482.122
lr = 0.100  |  7 loss: 29672.813
lr = 0.100  |  8 loss: 28460.208
lr = 0.100  |  9 loss: 27841.349
lr = 0.100  |  10 loss: 27123.731
lr = 0.100  |  11 loss: 27343.572
lr = 0.100  |  12 loss: 26184.523
lr = 0.100  |  13 loss: 25837.330
lr = 0.100  |  14 loss: 25678.197
lr = 0.100  |  15 loss: 25501.937
lr = 0.100  |  16 loss: 25345.974
lr = 0.100  |  17 loss: 24986.195
lr = 0.100  |  18 loss: 24811.637
lr = 0.100  |  19 loss: 24633.412
lr = 0.100  |  20 loss: 24531.259
lr = 0.100  |  21 loss: 24295.266
lr = 0.100  |  22 loss: 24068.456
lr = 0.100  |  23 loss: 24214.412
lr = 0.100  |  24 loss: 23944.265
lr = 0.100  |  25 loss: 23789.117
lr = 0.080  |  1 loss: 23349.915
lr = 0.080  |  2 loss: 23318.170
lr = 0.080  |  3 loss: 23273.535
lr = 0.080  |  4 loss: 23043.007
lr = 0.080  |  5 loss: 2325

## Test du réseau

In [45]:
correct = 0
total = 0


classes = ['Fold', 'Call', 'Raise']
class_correct = [0, 0, 0]
class_total = [1, 1, 1]


with torch.no_grad():
    for data in train_mains_3J_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = NN_3J(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print('Accuracy of the network : %d %%' % (100 * correct / total))

print('Accuracy for each class :')

for i in range(3):
    print('%s : %d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network : 83 %
Accuracy for each class :
Fold : 93 %
Call : 86 %
Raise : 57 %


## Sauvegarde du réseau 

In [47]:
model_ft = NN_3J.state_dict()
filename = 'model_theo_3J.pth'
torch.save(model_ft, filename)